In [23]:
import requests
import xml.etree.ElementTree as ET
from pyproj import Transformer
import json

# URL of the XML file
url = 'https://raw.githubusercontent.com/AzatSkyArchLab/ufa_urban_gov_data/main/MapPlan_c9fa34ae-f05d-4ad7-81ab-e430b75863a5.xml'

# Download the XML file
response = requests.get(url)
xml_content = response.content

# Parse the XML content
root = ET.fromstring(xml_content)

# Namespace required for finding elements
namespaces = {'spa1': 'urn://x-artefacts-rosreestr-ru/commons/complex-types/entity-spatial/1.0.2'}

# Initialize the transformer from MSK-02 (given PROJ string) to WGS 84 (EPSG:4326)
transformer = Transformer.from_proj(
    proj_from='+proj=tmerc +lat_0=0 +lon_0=55.03333333333 +k=1 +x_0=1300000 +y_0=-5409414.70 +ellps=krass +towgs84=23.57,-140.95,-79.8,0,0.35,0.79,-0.22 +units=m +no_defs',
    proj_to='epsg:4326',
    always_xy=True)

# Collect and transform coordinates
transformed_coordinates = []
for ordinate in root.findall('.//spa1:Ordinate', namespaces):
    x_msk02 = float(ordinate.get('Y'))
    y_msk02 = float(ordinate.get('X'))
    # Reproject the coordinates to geographic (lat, lon)
    lon_wgs84, lat_wgs84 = transformer.transform(x_msk02, y_msk02)
    transformed_coordinates.append([lon_wgs84, lat_wgs84])

# Ensure the polygon is closed by adding the first point at the end if it's not already
if transformed_coordinates[0] != transformed_coordinates[-1]:
    transformed_coordinates.append(transformed_coordinates[0])

# Create a GeoJSON polygon with the transformed coordinates
geojson_polygon = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [transformed_coordinates]
            },
            "properties": {
                "name": "Transformed Polygon in WGS 84"
            }
        }
    ]
}

# Print the GeoJSON
print(json.dumps(geojson_polygon, indent=4))
# Optionally, you can save this GeoJSON to a file
with open('transformed_polygon_WGS84.geojson', 'w') as f:
     json.dump(geojson_polygon, f, indent=4)


{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            56.08251476844403,
                            54.8236877561263
                        ],
                        [
                            56.08252396451898,
                            54.82368237633281
                        ],
                        [
                            56.08336850010091,
                            54.82137841432306
                        ],
                        [
                            56.0833254336875,
                            54.821373576255645
                        ],
                        [
                            56.08185552113443,
                            54.82120076696224
                        ],
                        [
                            56.0814